In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# ✅ Path update karo agar file location alag ho
path = "/content/drive/MyDrive/all_stocks_5yr.csv"
df = pd.read_csv(path)
print("✅ File loaded successfully!")
print(df.columns)

df["date"] = pd.to_datetime(df["date"])
df = df.set_index("date")
df = df.interpolate(method="time").bfill().ffill()

features = ["open", "high", "low", "close", "volume"]
df = df[features]

scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)

def create_sequences(data, seq_len=24):
    seqs = []
    for i in range(len(data) - seq_len):
        seqs.append(data[i:i+seq_len])
    return np.array(seqs)

sequences = create_sequences(scaled, 24)
X_train = sequences[:int(0.8*len(sequences))]

print("✅ Sequences shape:", X_train.shape)


In [ ]:
import tensorflow as tf

class SimpleTimeGAN(tf.keras.Model):
    def __init__(self, timesteps, features, hidden_dim=24, num_layers=3):
        super(SimpleTimeGAN, self).__init__()
        self.timesteps = timesteps
        self.features = features

        self.encoder = tf.keras.Sequential(
            [tf.keras.layers.LSTM(hidden_dim, return_sequences=True)] +
            [tf.keras.layers.LSTM(hidden_dim) for _ in range(num_layers - 1)]
        )
        self.generator = tf.keras.Sequential([
            tf.keras.layers.LSTM(hidden_dim, return_sequences=True),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(features, activation='sigmoid'))
        ])
        self.discriminator = tf.keras.Sequential([
            tf.keras.layers.LSTM(hidden_dim, return_sequences=False),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

    def compile(self, g_opt, d_opt):
        super(SimpleTimeGAN, self).compile()
        self.g_opt = g_opt
        self.d_opt = d_opt
        self.bce = tf.keras.losses.BinaryCrossentropy()

    @tf.function
    def train_step(self, real_seq):
        batch_size = tf.shape(real_seq)[0]
        noise = tf.random.normal((batch_size, self.timesteps, self.features))

        # Train discriminator
        with tf.GradientTape() as tape_d:
            fake_seq = self.generator(noise)
            real_pred = self.discriminator(real_seq)
            fake_pred = self.discriminator(fake_seq)
            d_loss = self.bce(tf.ones_like(real_pred), real_pred) + \
                     self.bce(tf.zeros_like(fake_pred), fake_pred)
        grads_d = tape_d.gradient(d_loss, self.discriminator.trainable_variables)
        self.d_opt.apply_gradients(zip(grads_d, self.discriminator.trainable_variables))

        # Train generator
        with tf.GradientTape() as tape_g:
            fake_seq = self.generator(noise)
            fake_pred = self.discriminator(fake_seq)
            g_loss = self.bce(tf.ones_like(fake_pred), fake_pred)
        grads_g = tape_g.gradient(g_loss, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(grads_g, self.generator.trainable_variables))

        return {"d_loss": d_loss, "g_loss": g_loss}
timesteps = X_train.shape[1]
features = X_train.shape[2]

model = SimpleTimeGAN(timesteps, features)
model.compile(
    g_opt=tf.keras.optimizers.Adam(1e-4),
    d_opt=tf.keras.optimizers.Adam(1e-4)
)

history = []
EPOCHS = 50
BATCH_SIZE = 128

for epoch in range(EPOCHS):
    for i in range(0, len(X_train), BATCH_SIZE):
        batch = X_train[i:i+BATCH_SIZE]
        metrics = model.train_step(batch)
    if epoch % 5 == 0:
        print(f"Epoch {epoch}: D_loss={metrics['d_loss']:.4f}, G_loss={metrics['g_loss']:.4f}")
noise = tf.random.normal((1, timesteps, features))
synthetic_data = model.generator(noise).numpy()

plt.figure(figsize=(10,5))
plt.plot(X_train[0,:,0], label="Real (Open)")
plt.plot(synthetic_data[0,:,0], '--', label="Synthetic (Open)")
plt.title("Real vs Synthetic (Open Feature)")
plt.legend()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# ✅ Step 1: Generate synthetic samples
num_samples = 1000
noise = tf.random.normal((num_samples, timesteps, features))
synthetic_data = model.generator(noise).numpy()

# Flatten for visualization
real_flat = X_train[:num_samples].reshape(num_samples, -1)
synthetic_flat = synthetic_data.reshape(num_samples, -1)

# ✅ Step 2: PCA (for dimensionality reduction)
pca = PCA(n_components=2)
pca_real = pca.fit_transform(real_flat)
pca_synth = pca.transform(synthetic_flat)

plt.figure(figsize=(8,6))
plt.scatter(pca_real[:,0], pca_real[:,1], alpha=0.5, label='Real', s=10)
plt.scatter(pca_synth[:,0], pca_synth[:,1], alpha=0.5, label='Synthetic', s=10)
plt.title("PCA Comparison: Real vs Synthetic Data")
plt.legend()
plt.show()

# ✅ Step 3: t-SNE visualization (for better structure)
combined = np.vstack([real_flat, synthetic_flat])
labels = np.array([0]*num_samples + [1]*num_samples)
tsne = TSNE(n_components=2, perplexity=40, learning_rate=200, n_iter=1000)
tsne_results = tsne.fit_transform(combined)

plt.figure(figsize=(8,6))
plt.scatter(tsne_results[labels==0,0], tsne_results[labels==0,1], label="Real", s=10, alpha=0.6)
plt.scatter(tsne_results[labels==1,0], tsne_results[labels==1,1], label="Synthetic", s=10, alpha=0.6)
plt.title("t-SNE Comparison: Real vs Synthetic Data")
plt.legend()
plt.show()

# ✅ Step 4: Save synthetic data to CSV
synthetic_df = pd.DataFrame(
    synthetic_data.reshape(-1, features),
    columns=[f"feature_{i+1}" for i in range(features)]
)

csv_path = "/content/drive/MyDrive/synthetic_timegan_data.csv"
synthetic_df.to_csv(csv_path, index=False)
print(f"✅ Synthetic data saved successfully at: {csv_path}")


In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# ✅ Step 1: Generate synthetic samples
num_samples = 1000
noise = tf.random.normal((num_samples, timesteps, features))
synthetic_data = model.generator(noise).numpy()

# ✅ Step 2: Flatten data for visualization
real_flat = X_train[:num_samples].reshape(num_samples, -1)
synthetic_flat = synthetic_data.reshape(num_samples, -1)

# ✅ Step 3: PCA (for 2D comparison)
pca = PCA(n_components=2)
pca_real = pca.fit_transform(real_flat)
pca_synth = pca.transform(synthetic_flat)

plt.figure(figsize=(8,6))
plt.scatter(pca_real[:,0], pca_real[:,1], alpha=0.5, label='Real', s=10)
plt.scatter(pca_synth[:,0], pca_synth[:,1], alpha=0.5, label='Synthetic', s=10)
plt.title("PCA Comparison: Real vs Synthetic Data")
plt.legend()
plt.show()

# ✅ Step 4: t-SNE visualization (for detailed structure)
combined = np.vstack([real_flat, synthetic_flat])
labels = np.array([0]*num_samples + [1]*num_samples)
tsne = TSNE(n_components=2, perplexity=40, learning_rate=200, n_iter=1000)
tsne_results = tsne.fit_transform(combined)

plt.figure(figsize=(8,6))
plt.scatter(tsne_results[labels==0,0], tsne_results[labels==0,1], label="Real", s=10, alpha=0.6)
plt.scatter(tsne_results[labels==1,0], tsne_results[labels==1,1], label="Synthetic", s=10, alpha=0.6)
plt.title("t-SNE Comparison: Real vs Synthetic Data")
plt.legend()
plt.show()

# ✅ Step 5: Save synthetic data with feature names
feature_names = ["open", "high", "low", "close", "volume"]

synthetic_df = pd.DataFrame(
    synthetic_data.reshape(-1, features),
    columns=feature_names
)

csv_path = "/content/drive/MyDrive/synthetic_timegan_data_named.csv"
synthetic_df.to_csv(csv_path, index=False)
print(f"✅ Synthetic data saved successfully at:\n📁 {csv_path}")

# ✅ Preview first few rows
print("\n🔹 Sample synthetic data preview:")
print(synthetic_df.head())


In [ ]:
!pip install dtaidistance


In [ ]:
from dtaidistance import dtw
print("✅ DTW module imported successfully!")


In [ ]:

# 1. Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# 2. Load Dataset from Drive
path = "/content/drive/MyDrive/all_stocks_5yr.csv"
df = pd.read_csv(path)
print("✅ File loaded successfully!")
print(df.head())

# 3. Preprocessing
df["date"] = pd.to_datetime(df["date"])      # Convert to datetime
df = df.set_index("date")                    # ✅ Set datetime index for time interpolation

# Handle missing values (interpolate + backfill)
df = df.interpolate(method="time").bfill().ffill()
print("✅ Missing values handled.")

# Select useful numeric columns
features = ["open", "high", "low", "close", "volume"]
df = df[features]
print("✅ Feature selection complete.")
print(df.head())

# Normalize (0–1 scaling)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)
print("✅ Data scaled between 0–1")

# Create sequences (e.g., 24 timesteps)
def create_sequences(data, timesteps=24):
    seqs = []
    for i in range(len(data) - timesteps):
        seqs.append(data[i:i+timesteps])
    return np.array(seqs)

timesteps = 24
X_train = create_sequences(scaled_data, timesteps)
print("✅ Sequences created:", X_train.shape)

# 4. Simple TimeGAN Model
class SimpleTimeGAN(tf.keras.Model):
    def __init__(self, timesteps, features, hidden_dim=24, num_layers=3):
        super(SimpleTimeGAN, self).__init__()
        self.timesteps = timesteps
        self.features = features

        self.encoder = tf.keras.Sequential(
            [tf.keras.layers.LSTM(hidden_dim, return_sequences=True)] +
            [tf.keras.layers.LSTM(hidden_dim) for _ in range(num_layers - 1)]
        )
        self.generator = tf.keras.Sequential([
            tf.keras.layers.LSTM(hidden_dim, return_sequences=True),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(features, activation='sigmoid'))
        ])
        self.discriminator = tf.keras.Sequential([
            tf.keras.layers.LSTM(hidden_dim, return_sequences=False),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

    def compile(self, g_opt, d_opt):
        super(SimpleTimeGAN, self).compile()
        self.g_opt = g_opt
        self.d_opt = d_opt
        self.bce = tf.keras.losses.BinaryCrossentropy()

    @tf.function
    def train_step(self, real_seq):
        batch_size = tf.shape(real_seq)[0]
        noise = tf.random.normal((batch_size, self.timesteps, self.features))

        # --- Train Discriminator ---
        with tf.GradientTape() as tape_d:
            fake_seq = self.generator(noise)
            real_pred = self.discriminator(real_seq)
            fake_pred = self.discriminator(fake_seq)
            d_loss = self.bce(tf.ones_like(real_pred), real_pred) + \
                     self.bce(tf.zeros_like(fake_pred), fake_pred)
        grads_d = tape_d.gradient(d_loss, self.discriminator.trainable_variables)
        self.d_opt.apply_gradients(zip(grads_d, self.discriminator.trainable_variables))

        # --- Train Generator ---
        with tf.GradientTape() as tape_g:
            fake_seq = self.generator(noise)
            fake_pred = self.discriminator(fake_seq)
            g_loss = self.bce(tf.ones_like(fake_pred), fake_pred)
        grads_g = tape_g.gradient(g_loss, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(grads_g, self.generator.trainable_variables))

        return {"d_loss": d_loss, "g_loss": g_loss}


# 5. Initialize Model
timesteps = X_train.shape[1]
features = X_train.shape[2]

model = SimpleTimeGAN(timesteps, features)
model.compile(
    g_opt=tf.keras.optimizers.Adam(1e-4),
    d_opt=tf.keras.optimizers.Adam(1e-4)
)

# 6. Train Model
EPOCHS = 50
BATCH_SIZE = 128
history = []

print("🏋️‍♂️ Training started... (this will take a few minutes)")

for epoch in range(EPOCHS):
    for i in range(0, len(X_train), BATCH_SIZE):
        batch = X_train[i:i+BATCH_SIZE]
        metrics = model.train_step(batch)
    history.append(metrics)
    if epoch % 5 == 0:
        print(f"Epoch {epoch:03d}: D_loss={metrics['d_loss']:.4f}, G_loss={metrics['g_loss']:.4f}")

print("✅ Training complete!")

# 7. Generate Synthetic Data
noise = tf.random.normal((1, timesteps, features))
synthetic_data = model.generator(noise).numpy()

# 8. Visualization: Real vs Synthetic
plt.figure(figsize=(10, 5))
plt.plot(X_train[0,:,0], label="Real (Open)")
plt.plot(synthetic_data[0,:,0], '--', label="Synthetic (Open)")
plt.title("📈 Real vs Synthetic Stock Data (Open Price)")
plt.legend()
plt.show()


In [ ]:

# 📊 9. Visualization & Analytics Section
import seaborn as sns
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# ---------- Create Synthetic Samples ----------
noise = tf.random.normal((100, X_train.shape[1], X_train.shape[2]))
synthetic_data = model.generator(noise).numpy()

# ✅ Match real and synthetic sample sizes
n_samples = min(len(X_train), len(synthetic_data))
real_subset = X_train[:n_samples]
synth_subset = synthetic_data[:n_samples]

# ✅ Define feature names
features = ["Open", "High", "Low", "Close", "Volume"]

# ---------- (a) Real vs Synthetic Overlay ----------
plt.figure(figsize=(10,5))
plt.plot(real_subset[0,:,0], label="Real (Open)")
plt.plot(synth_subset[0,:,0], '--', label="Synthetic (Open)")
plt.title("📈 Real vs Synthetic Stock Data (Open)")
plt.legend()
plt.show()

# ---------- (b) Distribution Comparison ----------
real_flat = real_subset.reshape(-1, X_train.shape[2])
synth_flat = synth_subset.reshape(-1, X_train.shape[2])

plt.figure(figsize=(12,6))
for i, feature in enumerate(features):
    plt.subplot(2,3,i+1)
    sns.kdeplot(real_flat[:, i], label="Real", fill=True)
    sns.kdeplot(synth_flat[:, i], label="Synthetic", fill=True, linestyle="--")
    plt.title(feature)
plt.suptitle("📊 Distribution Comparison: Real vs Synthetic Features")
plt.tight_layout()
plt.show()

# ---------- (c) Correlation Heatmaps ----------
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
sns.heatmap(pd.DataFrame(real_flat, columns=features).corr(), annot=True, cmap="coolwarm")
plt.title("Real Correlation")

plt.subplot(1,2,2)
sns.heatmap(pd.DataFrame(synth_flat, columns=features).corr(), annot=True, cmap="coolwarm")
plt.title("Synthetic Correlation")

plt.tight_layout()
plt.show()

# ---------- (d) Dynamic Time Warping (DTW) ----------
!pip install fastdtw > /dev/null
from fastdtw import fastdtw
dtw_distances = []
for i in range(50):
    d, _ = fastdtw(real_subset[i,:,0], synth_subset[i,:,0])
    dtw_distances.append(d)
avg_dtw = np.mean(dtw_distances)
print(f"📏 Average DTW Distance (lower = better): {avg_dtw:.4f}")

# ---------- (e) Maximum Mean Discrepancy (MMD) ----------
def compute_mmd(x, y, gamma=1.0):
    xx = rbf_kernel(x, x, gamma)
    yy = rbf_kernel(y, y, gamma)
    xy = rbf_kernel(x, y, gamma)
    return xx.mean() + yy.mean() - 2 * xy.mean()

mmd_score = compute_mmd(real_flat, synth_flat)
print(f"🧮 MMD Score (lower = better): {mmd_score:.6f}")

# ---------- (f) Predictive Utility ----------
X_real, y_real = real_subset[:,:-1,:], real_subset[:,-1,0]
X_syn, y_syn = synth_subset[:,:-1,:], synth_subset[:,-1,0]

X_real = X_real.reshape(len(X_real), -1)
X_syn = X_syn.reshape(len(X_syn), -1)

model_lr = LinearRegression().fit(X_syn, y_syn)
y_pred = model_lr.predict(X_real)
mse = mean_squared_error(y_real, y_pred)
r2 = r2_score(y_real, y_pred)
print(f"📉 Predictive Utility — MSE: {mse:.6f}, R²: {r2:.4f}")

# 📋 10. Summary Table
summary = pd.DataFrame({
    "Metric": ["DTW Distance", "MMD Score", "Predictive MSE", "Predictive R²"],
    "Value": [avg_dtw, mmd_score, mse, r2],
    "Interpretation": [
        "Measures shape similarity of temporal patterns",
        "Measures distribution similarity between real & synthetic data",
        "Tests generalization from synthetic→real forecasting",
        "Higher = better predictive realism"
    ]
})

print("\n📊 Quantitative Evaluation Summary")
display(summary)
